<a href="https://colab.research.google.com/github/HunEee/OAuth2/blob/master/%EC%B5%9C%EC%A2%85%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 깃 레퍼지토리 가져오기
!git clone https://github.com/YoungjinBD/data.git

Cloning into 'data'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 286 (delta 100), reused 23 (delta 23), pack-reused 98 (from 1)
Receiving objects: 100% (286/286), 833.62 KiB | 10.55 MiB/s, done.
Resolving deltas: 100% (112/112), done.


# 1유형

In [5]:
# 기본 내용
import pandas as pd
import numpy as np

# 파일 데이터 프레임으로 가져오기
df = pd.read_csv('data/exam/9_1_1.csv')

# 컬럼 끼리 계산해서 새로운 컬럼을 만듦
df['총대출액'] = df['금액1']+df['금액2']

# 데이터프레임 비율로 새로운 데이터 프레임 생성 가능
df_gum = df[df['구분']=="검거건수"].set_index("연도").drop(columns="구분")
df_bal = df[df['구분']=="발생건수"].set_index("연도").drop(columns="구분")
df_ratio = df_gum/df_bal

# 열을 녹인다. -> id_vars 유지할 열, var_name 녹이는 열들의 컬럼이다. -> 아래는 연도,범죄유형,검거건수로 구성
df_melt = df_gum.reset_index().melt(id_vars='연도',var_name='범죄유형',value_name='검거건수')

# 열의 최대값을 시리즈로 반환
df_max = df_ratio.idxmax(axis=1)

# 시리즈의 열 이름을 지정해주기
df_max.name = '범죄유형'

# 인덱스를 번호로 초기화
df_max_reset = df_max.reset_index()

# 조인
df_final= pd.merge(df_max_reset, df_melt, on=['연도','범죄유형'],how='left')


KeyError: '구분'

In [ ]:
# 문자열과 날짜 변환

# 포함문자 찾기
ma = df['STATION_ADDR1'].str.contains('마포구', na=False)

# 문자열을 나눠서 칼럼으로 만드는 옵션 expand
df[['연도','월']] = df['기간'].str.split('_', expand=True)

# 월을 없애버리고 정수형으로 전
df['월']=df['월'].str.replace('월','').astype(int)

# right는 오른쪽을 포함(0< x =<3 , 3< x =<6)
df['분기'] = pd.cut(df['월'], bins=[0,3,6,9,12], labels = [1,2,3,4], right=True)
df['gubun'] = pd.cut(df['bmi'], bins=[0,18.5,23,25,float('inf')],labels = ['저체중','정상','과체중','비만'], right=False)

##############################################################################
# 변경 칼럼 목록
cols = ['신고일자','신고시각','처리일자','처리시각']

# 컬럼 문자열로 변환 (dtype 충돌 없음)
df[cols] = df[cols].astype(str)

# 날짜는 8자리, 시간은 6자리로 패딩
df['신고일자'] = df['신고일자'].str.zfill(8)
df['신고시각'] = df['신고시각'].str.zfill(6)
df['처리일자'] = df['처리일자'].str.zfill(8)
df['처리시각'] = df['처리시각'].str.zfill(6)

# 정상적인 datetime 생성
df['신고일시'] = pd.to_datetime(df['신고일자'] + df['신고시각'], format='%Y%m%d%H%M%S')
df['처리일시'] = pd.to_datetime(df['처리일자'] + df['처리시각'], format='%Y%m%d%H%M%S')

# 처리시간을 초단위로 생성
df['처리시간'] = (df['처리일시'] - df['신고일시']).dt.total_seconds()

# 컬 조합으로 날짜 컬럼 만들기
df['date'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day']))
# 이 형태로 됨
cond1 = df['date'] >= '2016-09-01'


In [ ]:
# 집계하기
dfgender = df.groupby(['year','gender','지역코드'])['총대출액'].sum().reset_index()

# 평균을 집계한 데이터 최대값
df1=df.groupby(['대륙'])['맥주소비량'].mean().idxmax()

# 그룹바이에 유용한 함수
len(df['국가'].unique())

In [ ]:
# 피벗 활용하기(9회 1유형 참고)
df_pivot = dfgender.pivot_table(index=['year','지역코드'],columns='gender', values='총대출액',fill_value=0, aggfunc='sum' )

# 피벗에서 새로운 열을 만들 수 있다.
df_pivot['abs_diff'] = abs(df_pivot[0]-df_pivot[1])

# 피벗에서 인덱스 가져오기 -> 중첩인덱스
df_pivot.index[0][1]

# 피벗한 것에서 가장 큰 값을 가져오는 함
max_idx = df_pivot['abs_diff'].idxmax()

# 튜플을 year, 지역코드로 분해
max_year, max_region = max_idx

# 피벗을 정렬하기
df_pivot_sort = df_pivot.sort_values(by='abs_diff', ascending=False)

In [ ]:
# 결측치 대체
df3 = pd.read_csv('data/exam/9_1_3.csv')

# 데이터 확인, 결측치 확인
# print(df3.head(),df3.info())
# print(df3.isna().sum())

df_mean = df3['평균만족도'].mean()

df3['평균만족도'] =df3['평균만족도'].fillna(df_mean)


# 결측치 대체 (부서+등급별 평균 근속연수, 소수점 버림)
# transform 이 핵심 코드
df3['근속연수'] = df3['근속연수'].fillna(
    df3.groupby(['부서','등급'])['근속연수']
       .transform('mean')  # 그룹별 평균
       .floordiv(1)         # 소수점 버림
)


In [ ]:
# 결측치 제거
df.dropna(inplace=True)

# 결측치과 최대인 컬럼 찾기
df.isnull().sum().idxmax()

In [ ]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df = df.loc[:,['CO(GT)', 'NMHC(GT)']]
df_scaled = scaler.fit_transform(df)
print(df_scaled)

# 스케일링 이후는 넘파이 형태라 loc나 iloc를 쓸 수 없고 순서로 가져와야함
co_std = df_scaled[:,0].std().round(2)
nmhc_std = df_scaled[:,1].std().round(2)


In [ ]:
#IQR
q3 = df1['conventional'].astype(int).quantile(0.75)
q1 = df1['conventional'].astype(int).quantile(0.25)
iqr = q3-q1

In [ ]:
# 상위 70퍼 추출
top_70 = int(len(df)*0.7)
df1 = df.iloc[:347]

In [ ]:
# loc 함수와 조건
cond1 = df3['부서']=="HR"
cond2 = df3['등급']=="A"

mean_kun = df3[cond1&cond2]['근속연수'].mean()

print(mean_kun)

mean_kun2 = df3.loc[cond1&cond2,'근속연수'].mean()
print(mean_kun2)


In [ ]:
# help 함수 활용법

import pandas as pd
# help(pd.DataFrame.pivot_table)
help(pd.DataFrame.melt)

Help on function pivot_table in module pandas.core.frame:

pivot_table(self, values=None, index=None, columns=None, aggfunc: 'AggFuncType' = 'mean', fill_value=None, margins: 'bool' = False, dropna: 'bool' = True, margins_name: 'Level' = 'All', observed: 'bool | lib.NoDefault' = <no_default>, sort: 'bool' = True) -> 'DataFrame'
    Create a spreadsheet-style pivot table as a DataFrame.

    The levels in the pivot table will be stored in MultiIndex objects
    (hierarchical indexes) on the index and columns of the result DataFrame.

    Parameters
    ----------
    values : list-like or scalar, optional
        Column or columns to aggregate.
    index : column, Grouper, array, or list of the previous
        Keys to group by on the pivot table index. If a list is passed,
        it can contain any of the other types (except list). If an array is
        passed, it must be the same length as the data and will be used in
        the same manner as column values.
    columns : column, G

# 2유형

In [37]:
################################################################################
# X와 Y 데이터 확인
train =  pd.read_csv('data/exam/7_2_train.csv')
test =  pd.read_csv('data/exam/7_2_test.csv')

#print(train.head())

################################################################################
# X와 Y 데이터 확인
train = train.drop(['ID','level_0'],axis=1)
test = test.drop(['ID','level_0'],axis=1)


train_x = train.drop(['Target'],axis=1)
train_y = train['Target']

test_x = train.drop(['Target'],axis=1)
test_y = train['Target']


In [38]:
################################################################################
# 데이터 분리하기
from sklearn.model_selection import train_test_split

x_tr, x_val,y_tr,y_val = train_test_split(train_x,train_y,test_size=0.3,random_state=1)

# 확인
print(x_tr.shape,x_val.shape,y_tr.shape,y_val.shape)
################################################################################
# 결측치가 있으면 원핫 인코딩 전에 대체
# 결측값 확인
#print(x_tr.isnull().sum())
#print(x_val.isnull().sum())

gender_mode = x_tr['Gender'].mode()[0]

x_tr['Gender'].fillna(gender_mode, inplace=True)
x_val['Gender'].fillna(gender_mode, inplace=True)
test_x['Gender'].fillna(gender_mode, inplace=True)

median_val = x_tr['age'].median()

x_tr['age'].fillna(median_val, inplace=True)
x_val['age'].fillna(median_val, inplace=True)
test_x['age'].fillna(median_val, inplace=True)

################################################################################
# 범주형변수 원핫 인코딩 -> 모든 x들에 인코딩해준다.
from sklearn.preprocessing import OneHotEncoder

# 범주형 변수(리스트로 만들기)
cat_columns = x_tr.select_dtypes('object').columns.to_list()
num_columns = x_tr.select_dtypes('number').columns.to_list()

# handle_unknown='ignore' → 학습에 없는 범주 등장 시 오류 방지
# sparse_output=False → numpy array 반환
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 모든 X데이터에 transfrom() , x_tr에만 fit_transform()
x_tr_cat = ohe.fit_transform(x_tr[cat_columns])
x_val_cat = ohe.transform(x_val[cat_columns])
test_x_cat = ohe.transform(test_x[cat_columns])

################################################################################
# 인코딩한 범주형변수와 수치형변수의 컬럼을 합친다.
x_tr_num = x_tr[num_columns]
x_val_num = x_val[num_columns]
test_x_num = test_x[num_columns]

x_tr_pre = np.concatenate([x_tr_cat,x_tr_num],axis=1)
x_val_pre = np.concatenate([x_val_cat,x_val_num],axis=1)
test_x_pre = np.concatenate([test_x_cat,test_x_num],axis=1)

(216, 16) (93, 16) (216,) (93,)


In [39]:
################################################################################
# 모델 적합

# 분류
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1)
rf.fit(x_tr_pre,y_tr)


# 수치
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=1)
rf.fit(x_tr,y_tr)


RandomForestClassifier(random_state=1)

In [40]:
################################################################################
# 모델 평가
pred = rf.predict(x_val_pre)

# f1_score -> 분류형 예측
from sklearn.metrics import f1_score
print(f1_score(y_val,pred, average = 'macro'))

# roc_auc_score -> 분류 예측**************** proba 사용, pred[:,1] -> 두번째 컬럼을 사용
from sklearn.metrics import roc_auc_score
pred = rf.predict_proba(x_val)
result = roc_auc_score(y_val,pred[:,1])


# mean_absolute_error -> 수치형 예측
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_val,pred))

# mean_squared_error -> 수치형 예
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_val,pred)
rmse = np.sqrt(mse)


################################################################################
# 정답 제출
pred_test = rf.predict(test_x_pre)
result = pd.DataFrame({'pred':pred_test})
result.to_csv('t.csv',index = False)

0.6128970106561143


ValueError: could not convert string to float: 'Graduate'

In [23]:
import sklearn.metrics
print([f for f in dir(sklearn.metrics) if f.startswith('mean')])

['mean_absolute_error', 'mean_absolute_percentage_error', 'mean_gamma_deviance', 'mean_pinball_loss', 'mean_poisson_deviance', 'mean_squared_error', 'mean_squared_log_error', 'mean_tweedie_deviance']


In [ ]:
# 범주형 변수 인코딩 -> 라벨 인코딩을 쓰면 수치형변수와 합치는 작업이 필요 없
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(x_tr['Gender'])
x_tr['Gender'] = le.transform(x_tr['Gender'])
x_val['Gender'] = le.transform(x_val['Gender'])
test_x['Gender'] = le.transform(test_x['Gender'])

# 3유형

In [ ]:
# 다중회귀분석
from statsmodels.formula.api import ols

# 포뮬러는 "종속변수 ~ 독립변수들의 합"
formula = "design ~ tenure + f2 + f3 + f4 + f5"
model = ols(formula, data = train).fit()

p_values = model.pvalues

# 유의하지 않은 칼럼 개수 -> 절편을 제외하기 위해 [1:]
non_sig_vars = (p_values[1:]>=0.05).sum()




In [45]:
# 로지스틱 회귀 분석(임곗값 0.5 기준)
# [2] 어느 회사가 조사한 고객 데이터는 100개의 샘플로 이루어져 있으며,
# 각 샘플에는 고객의 나이, 소득, 가족 수,그리고 제품 구매 여부가 포함되어 있다.
# 로지스틱 회귀 분석을 통해 고객의 제품 구매 여부를 예측 하고자 한다.

import pandas as pd
import numpy as np
train_data = pd.read_csv('data/exam/7_3_2_train.csv')
test_data = pd.read_csv('data/exam/7_3_2_test.csv')
#print(train_data.info(),test_data.info())

# ① 로지스틱 회귀 분석을 수행하고, 소득 변수의 오즈비를 계산하시오.
import statsmodels.api as sm

# 독립변수와 종속변수 설정(train 데이터 사용)
x_train = train_data[['age','income','family_members']]
y_train = train_data['purchase']

x_train = sm.add_constant(x_train)  # 상수항 추가
model = sm.Logit(y_train,x_train).fit()
print(model.summary())
odds_r = np.exp(model.params[1:])
print('1번 답:', round(odds_r['income'],3))

# ② train 데이터 기준의 잔차 이탈도(Residual Deviance)를 계산하시오.
residual = -2*model.llf
print('2번 답: ', round(residual,2) )

# ③ test 데이터로 오분류율을 계산하시오.
x_test = test_data[['age','income','family_members']]
y_test = test_data['purchase']

# 예측 수행 -> y_test와 같은 형태로 바꿔준다.
x_test = sm.add_constant(x_test) # 상수항 추가
prob = model.predict(x_test)
print(prob)
pred = (prob>=0.5).astype(int)
print(y_test)

# 오류율 계산
error_rate = np.mean(pred != y_test)
print('3번 답: ', round(error_rate,3))

Optimization terminated successfully.
         Current function value: 0.676174
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:               purchase   No. Observations:                   80
Model:                          Logit   Df Residuals:                       76
Method:                           MLE   Df Model:                            3
Date:                Thu, 27 Nov 2025   Pseudo R-squ.:                 0.02405
Time:                        13:11:44   Log-Likelihood:                -54.094
converged:                       True   LL-Null:                       -55.427
Covariance Type:            nonrobust   LLR p-value:                    0.4461
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.9261      1.127     -0.821      0.411      -3.136       1.284
age              

In [41]:
# ttest

#대응표본 t-검정 -> 학습 전후의 점수 차이가 유의미한 지 검정
from scipy.stats import ttest_rel
t_stat , p_value = ttest_rel(df['before'],df['after'])

# 독립 2표본 t-검정 -> 두 부서의 평균 근무 시간 차이를 검정
from scipy.stats import ttest_ind
static, p_value = ttest_ind(df[df['department']=='A']['hours_worked'],df[df['department']=='B']['hours_worked'])

# 일표본 t-검정 -> 모평균 75를 기준
from scipy.stats import ttest_1samp
stats, p_value = ttest_1samp(df['score'],popmean=75)


In [ ]:
# ANOVA 검정
# 세 캠페인의 평균 만족도 점수가 유의미하게 다른지 검정하기 위해 ANOVA 검정을 수행
from scipy.stats import f_oneway

stat,p_value =f_oneway(
    df[df['campaign']=='A']['satisfaction_score'],
    df[df['campaign']=='B']['satisfaction_score'],
    df[df['campaign']=='C']['satisfaction_score']
)

In [ ]:
# 크루스칼 왈리스
# 두 부서의 평균 근무 시간이 유의미하게 다른지 검정
from scipy.stats import kruskal

x=df[df['factory'] == 'A']['quality_score']
y=df[df['factory'] == 'B']['quality_score']
z=df[df['factory'] == 'C']['quality_score']

stat, p_value =kruskal(x, y,z)

In [ ]:
df = pd.read_csv('data/exam/3_3_2.csv')
print(df.head())

# ① 성별과 동아리 가입 여부 간의 독립성을 검정하기 위해 카이제곱 검정을 수행하고 검정통계량을 계산하시오.
# (소수 점 둘째 자리까지 반올림, 연속성 수정 미적용)
from scipy.stats import chi2_contingency
# print(help(chi2_contingency))

# 교차표 생성 (성별 × 동아리 가입 여부)
obs = pd.crosstab(df['gender'], df['club_membership'])

# print(obs)

# 카이제곱 검정 수행
stat, p, dof, expected = chi2_contingency(obs)

print("Chi-square:", stat)
print("p-value:", p)
# print("degrees of freedom:", dof)
# print("Expected frequencies:\n", expected)

In [ ]:
# 피어슨 상관계수
from scipy.stats import pearsonr

# 예측값과 실제값의 피어슨 상관계수
y_pred = model.predict(train)
y_true = train['design']

corr, _ = pearsonr(y_pred, y_true)
print('2번답: ',corr)

In [ ]:
# 윌콕슨
# 도입 전후의 업무처리 시간 차이가 유의미한 지 부호 순위 검정을 실시하고, 검정통계량을 계산하시오. (소수점 둘째 자리까지 반올림)
from scipy.stats import wilcoxon

# 부호 순위검정 수행
sta,p_value = wilcoxon(df['before'],df['after'])
print('2번 답: ',round(sta,2)  )


In [ ]:
# ② 직원들의 업무 수행 시간이 정규분포를 따르는 지 K-S 검정을 실시하고, 검정통계량을 계산하시오.
from scipy.stats import kstest, norm

avg_1 = round(df['work_hours'].mean(),3)
std_1 = round(df['work_hours'].std(),3)

static,p_value = kstest(df['work_hours'], 'norm', args=(avg_1,std_1))
print('2번 답: ',round(static,3))

In [ ]:
#  RMSE 구하기
from sklearn.metrics import root_mean_squared_error

y_pred_test = model.predict(test)

# 실제값과 예측값을 넣어서 평가
rmse = root_mean_squared_error(test['design'], y_pred_test)

####################################################################
#R2 구하기
from sklearn.metrics import r2_score

# 테스트 데이터 준비
x_test = test[['hours','age','experience']]
y_test = test['productivity']

# 포뮬러 방식에서는 상수항 자동 포함
x_pred = model.predict(x_test)  # test DataFrame 그대로 전달

# R² 계산
r2 = r2_score(y_test, x_pred)
